# Integrate Modern Data Architectures with Generative AI and interact using prompts for querying SQL databases & APIs

This notebook demonstrates how **large language models, such as Amazon Titan and Claude Anthropic, accessible via [Amazon BedRock](https://aws.amazon.com/bedrock/)** interact with AWS databases, data stores, and third-party data warehousing solutions like Snowflake. We showcase this interaction 1) by generating and running SQL queries, and 2) making requests to API endpoints. We achieve all of this by using the LangChain framework, which allows the language model to interact with its environment and connect with other sources of data. The LangChain framework operates based on the following principles: calling out to a language model, being data-aware, and being agentic. 

This notebook focuses on establishing connection to one data source, consolidating metadata, and returning fact-based data points in response to user queries using LLMs and LangChain. The solution can be enhanced to add multiple data sources.


<img src='images/img-genai-sql-langchain.png' width="800" height="600">


### Pre-requisites:
1. Use kernel Base Python 3.0.
2. Install the required packages.
3. Run the One time Setup by entering the user input parameters, copying the dataset, setup IAM role and finally run the crawler.
3. Access to the LLM API. In this notebook, Anthropic Model is used. Refer [here](https://console.anthropic.com/docs/access) for detais on how to get access to Anthropic API key.

**Note - This notebook was tested on kernel - conda_python3 in Region us-east-1**

1. Attach AmazonAthenaFullAccess, AWSGlueServiceRole in IAM.
2. Add the following custom policy in IAM to grant creating policy (double click cell to get json format).

{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "iam:AttachRolePolicy",
                "iam:CreateRole",
                "iam:CreatePolicy",
                "iam:GetRole",
                "iam:PassRole"
            ],
            "Effect": "Allow",
            "Resource": "*"
        }
    ]
}

### Solution Walkthrough:

Step 1. Connection to S3 through which LLMs can talk to your data. These channels include:
    - S3/Athena - to connect to the SageMaker's offline feature store on claims information. 
    
Step 2. Usage of Dynamic generation of prompt templates by populating metadata of the tables using Glue Data Catalog(GDC) as context. GDC was populated by running a crawler on the databases. Refer to the information here to create and run a glue crawler. In case of api, a line item was created in GDC data extract.

Step 3. Define Functions to 1/ determine the best data channel to answer the user query, 2/ Generate response to  user query

Step 4. Apply user query to LLM and Langchain to determine the data channel. After determining the data channel, run the Langchain SQL Database chain to convert 'text to sql' and run the query against the source data channel. 

Finally, display the results.


In [ ]:
!python3 -m pip install boto3-1.26.142-py3-none-any.whl --quiet

In [ ]:
!python3 -m pip install botocore-1.29.142-py3-none-any.whl --quiet

In [ ]:
%%writefile requirements.txt
sqlalchemy==1.4.47
snowflake-sqlalchemy
langchain==0.0.190
sqlalchemy-aurora-data-api
PyAthena[SQLAlchemy]==2.25.2
anthropic
redshift-connector==2.0.910
sqlalchemy-redshift==0.8.14
snowflake
streamlit
streamlit-chat 

In [ ]:
!pip install -r requirements.txt --quiet

In [ ]:
import json
import boto3

import sqlalchemy
from sqlalchemy import create_engine
# from snowflake.sqlalchemy import URL

from langchain.docstore.document import Document
from langchain import PromptTemplate,SagemakerEndpoint,SQLDatabase, SQLDatabaseChain, LLMChain
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import SQLDatabaseSequentialChain

from langchain.chains.api.prompt import API_RESPONSE_PROMPT
from langchain.chains import APIChain
from langchain.prompts.prompt import PromptTemplate
from langchain.chat_models import ChatAnthropic
from langchain.chains.api import open_meteo_docs

from typing import Dict
import time

In [ ]:
import boto3

### One Time Setup
Some of the resources needed for this notebook such as the IAM policy, AWS Glue database and Glue crawler are created through a cloud formation template. The next block of code does the setup based on user inputs.

**NOTE - The next two blocks of code need to be run only for the first time.**

### User Input


In [ ]:
# Provide user input
glue_databucket_name = 'sagemaker-studio-741094476554-9zkt2s8krvb' #Create this bucket in S3
glue_db_name='ihmnick-bankadditional'
glue_role=  'ihmnick-AWSGlueServiceRole-glueworkshop120'
glue_crawler_name=glue_db_name+'-crawler120'

### Create IAM Role that runs the crawler

In [ ]:
import boto3
import os
# Retrieve the AWS account number
sts_client = boto3.client('sts')
account_number = sts_client.get_caller_identity().get('Account')
# Retrieve the AWS region
#region = os.environ['AWS_REGION']
region = boto3.session.Session().region_name
print("AWS Account Number:", account_number)
print("AWS Region:", region)
trust_policy="""{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "",
      "Effect": "Allow",
      "Principal": {
        "Service": "glue.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}"""
managed_policy="""{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "glue:*"
            ],
            "Resource": [
                "arn:aws:glue:"""+region+""":"""+account_number+""":catalog",
                "arn:aws:glue:"""+region+""":"""+account_number+""":database/*",
                "arn:aws:glue:"""+region+""":"""+account_number+""":table/*"
            ],
            "Effect": "Allow",
            "Sid": "Readcrawlerresources"
        },
        {
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": [
                "arn:aws:glue:"""+region+""":"""+account_number+""":log-group:/aws-glue/crawlers*",
                "arn:aws:logs:*:*:/aws-glue/*",
                "arn:aws:logs:*:*:/customlogs/*"
            ],
            "Effect": "Allow",
            "Sid": "ReadlogResources"
        },
    {
            "Action": [
                "s3:PutObject",
                "s3:GetObject",
                "s3:PutBucketLogging",
                "s3:ListBucket",
                "s3:PutBucketVersioning"
            ],
            "Resource": [
                "arn:aws:s3:::"""+glue_databucket_name+"""",
                "arn:aws:s3:::"""+glue_databucket_name+"""/*"
            ],
            "Effect": "Allow",
            "Sid": "ReadS3Resources"
        }
    ]
    }"""
print(managed_policy, file=open('managed-policy.json', 'w'))
print(trust_policy, file=open('trust-policy.json', 'w'))

In [ ]:
%%sh -s "$glue_role" 
echo $1 
glue_role="$1"
managed_policy_name="managed-policy-$glue_role"
echo $managed_policy_name
aws iam create-role --role-name $glue_role --assume-role-policy-document file://trust-policy.json
output=$(aws iam create-policy --policy-document file://managed-policy.json --policy-name $managed_policy_name)
arn=$(echo "$output" | grep -oP '"Arn": "\K[^"]+')
echo "$arn"
aws iam attach-role-policy --policy-arn $arn --role-name $glue_role

In [ ]:
import boto3

client = boto3.client('glue')

# Create database 
try:
    response = client.create_database(
        DatabaseInput={
            'Name': glue_db_name,
            'Description': 'This database is created using Python boto3',
        }
    )
    print("Successfully created database")
except:
    print("error in creating database. Check if the database already exists")

#introducing some lag for the iam role to create
time.sleep(20) 

# Create Glue Crawler 
try:

    response = client.create_crawler(
        Name=glue_crawler_name,
        Role=glue_role,
        DatabaseName=glue_db_name,
        Targets={
            'S3Targets': [
                {
                    'Path': 's3://{BUCKET_NAME}/bank-additional/'.format(BUCKET_NAME =glue_databucket_name)
                }
            ]
        },
        TablePrefix=''
    )
    
    print("Successfully created crawler")
except:
    print("error in creating crawler. However, if the crawler already exists, the crawler will run.")

# Run the Crawler
try:
    response = client.start_crawler(Name=glue_crawler_name )
    print("Successfully started crawler. The crawler may take 2-5 mins to detect the schema.")
    while True:
        # Get the crawler status
        response = client.get_crawler(Name=glue_crawler_name)
         # Extract the crawler state
        status = response['Crawler']['State']
        # Print the crawler status
        print(f"Crawler '{glue_crawler_name}' status: {status}")
        if status == 'READY':  # Replace 'READY' with the desired completed state
            break  # Exit the loop if the desired state is reached

        time.sleep(10)  # Sleep for 10 seconds before checking the status again
    
except:
    print("error in starting crawler. Check the logs for the error details.")

Before proceeding to the next step, check the status of the crawler. It should change from RUNNING to READY. 

### Step 1 - Connect to databases using SQL Alchemy. 

Under the hood, LangChain uses SQLAlchemy to connect to SQL databases. The SQLDatabaseChain can therefore be used with any SQL dialect supported by SQLAlchemy, 
such as MS SQL, MySQL, MariaDB, PostgreSQL, Oracle SQL, and SQLite. Please refer to the SQLAlchemy documentation for more information about requirements for connecting to your database. 


**Important**: The code below establishes a database connection for data sources and Large Language Models. Please note that the solution will only work if the database connection for your sources is defined in the cell below. Please refer to the Pre-requisites section. If your use case requires data from Aurora MySQL alone, then please comment out other data sources. Furthermore, please update the cluster details and variables for Aurora MySQL accordingly.

In [ ]:
# Define connections

# Collect credentials from Secrets Manager
# Refer here on how to use AWS Secrets Manager - https://docs.aws.amazon.com/secretsmanager/latest/userguide/intro.html
# client = boto3.client('secretsmanager')
client = boto3.client('glue')
region=client.meta.region_name


#LLM 
#get the llm api key
#llm variables
#Refer here for access to Anthropic API Keys https://console.anthropic.com/docs/access
anthropic_secret_id = "anthropic"#<your anthropic secret id>
## llm get credentials from secrets manager
response = client.get_secret_value(SecretId=anthropic_secret_id)
secrets_credentials = json.loads(response['SecretString'])
ANTHROPIC_API_KEY = secrets_credentials['ANTHROPIC_API_KEY']
#define large language model here. Make sure to set api keys for the variable ANTHROPIC_API_KEY
llm = ChatAnthropic(temperature=0, anthropic_api_key=ANTHROPIC_API_KEY, max_tokens_to_sample = 512)



# Connect to S3 using Athena

# Athena variables
connathena = f"athena.{region}.amazonaws.com" 
portathena = '443' #Update, if port is different
schemaathena = glue_db_name #from user defined params
s3stagingathena = f's3://{glue_databucket_name}/athenaresults/'#from cfn params
wkgrpathena = 'primary'#Update, if workgroup is different
# tablesathena=['dataset']#[<tabe name>]

# Create the athena connection string
connection_string = f"awsathena+rest://@{connathena}:{portathena}/{schemaathena}?s3_staging_dir={s3stagingathena}/&work_group={wkgrpathena}"

# Create the athena SQLAlchemy engine
engine_athena = create_engine(connection_string, echo=False)
dbathena = SQLDatabase(engine_athena)

gdc = [schemaathena] 

In [ ]:
gdc

### Step 2 - Generate Dynamic Prompt Templates
Build a consolidated view of Glue Data Catalog by combining metadata stored for all the databases in pipe delimited format.

In [ ]:
#Generate Dynamic prompts to populate the Glue Data Catalog
#harvest aws crawler metadata

def parse_catalog():
    #Connect to Glue catalog
    #get metadata of redshift serverless tables
    columns_str=''
    
    #define glue cient
    glue_client = boto3.client('glue')
    
    for db in gdc:
        response = glue_client.get_tables(DatabaseName =db)
        for tables in response['TableList']:
            #classification in the response for s3 and other databases is different. Set classification based on the response location
            if tables['StorageDescriptor']['Location'].startswith('s3'):  classification='s3' 
            else:  classification = tables['Parameters']['classification']
            for columns in tables['StorageDescriptor']['Columns']:
                    dbname,tblname,colname=tables['DatabaseName'],tables['Name'],columns['Name']
                    columns_str=columns_str+f'\n{classification}|{dbname}|{tblname}|{colname}'                     
    #API
    ## Append the metadata of the API to the unified glue data catalog
    columns_str=columns_str+'\n'+('api|meteo|weather|weather')
    return columns_str

glue_catalog = parse_catalog()

#display a few lines from the catalog
print('\n'.join(glue_catalog.splitlines()[-10:]) )


### Step 3 - Define Functions to 1/ determine the best data channel to answer the user query, 2/ Generate response to  user query

In this code sample, we use the Anthropic Model to generate inferences. You can utilize SageMaker JumpStart models  to achieve the same. 
Guidance on how to use the JumpStart Models is available in the notebook - mda_with_llm_langchain_smjumpstart_flant5xl

In [ ]:
#Function 1 'Infer Channel'
#define a function that infers the channel/database/table and sets the database for querying
def identify_channel(query):
    db = {}
    #Prompt 1 'Infer Channel'
    ##set prompt template. It instructs the llm on how to evaluate and respond to the llm. It is referred to as dynamic since glue data catalog is first getting generated and appended to the prompt.
    prompt_template = """
     From the table below, find the database (in column database) which will contain the data (in corresponding column_names) to answer the question 
     {query} \n
     """+glue_catalog +""" 
     Give your answer as database == 
     Also,give your answer as database.table == 
     """
    ##define prompt 1
    PROMPT_channel = PromptTemplate( template=prompt_template, input_variables=["query"]  )

    # define llm chain
    llm_chain = LLMChain(prompt=PROMPT_channel, llm=llm)
    #run the query and save to generated texts
    generated_texts = llm_chain.run(query)
    print('identified channel:', generated_texts)

    #set the channel from where the query can be answered
    if 'database' in generated_texts: 
            channel='db'
            db=dbathena
            print("SET database to athena")
    elif 'api' in generated_texts: 
            channel='api'
            print("SET database to weather api")        
    else: raise Exception("User question cannot be answered by any of the channels mentioned in the catalog")
    print("Step complete. Channel is: ", channel)
    
    return channel, db

#Function 2 'Run Query'
#define a function that infers the channel/database/table and sets the database for querying
def run_query(query):

    channel, db = identify_channel(query) #call the identify channel function first
        
    ##Prompt 2 'Run Query'
    #after determining the data channel, run the Langchain SQL Database chain to convert 'text to sql' and run the query against the source data channel. 
    #provide rules for running the SQL queries in default template--> table info.

    _DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.

    Do not append 'Query:' to SQLQuery.
    
    Display SQLResult after the query is run in plain english that users can understand. 

    Provide answer in simple english statement.
 
    Only use the following tables:

    {table_info}

    Question: {input}"""

    PROMPT_sql = PromptTemplate(
        input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
    )

    
    if channel=='db':
        db_chain = SQLDatabaseChain.from_llm(llm, db, prompt=PROMPT_sql, verbose=True, return_intermediate_steps=False)
        response=db_chain.run(query)
    elif channel=='api':
        chain_api = APIChain.from_llm_and_api_docs(llm, open_meteo_docs.OPEN_METEO_DOCS, verbose=True)
        response=chain_api.run(query)
    else: raise Exception("Unlisted channel. Check your unified catalog")
    return response



### Step 4 - Run the run_query function that in turn calls the Langchain SQL Database chain to convert 'text to sql' and runs the query against the source data channel

Some samples are provided below for test runs. Uncomment the query to run.

In [ ]:
# Enter the query
## Few queries to try out - 
# query = """How many people are married?""" 
# query = """What is the maximum age?""" 
# query = """What percentage of customers in each age group enroll for CDs?"""
query = """What is the enrollment rate for each marital status?"""
# query = """What percentage of customers in each education level enroll for CDs?"""
# query = """What percentage of customers with existing loans (housing loan, personal loan) enroll for CDs?"""
# query = """What is the average enrollment rate over time for each month/quarter?"""
# query = """What is the average enrollment rate at different time periods after the last marketing contact (e.g. 0-7 days, 8-14 days, 15-30 days, 30+ days)?"""
# query = """What is the enrollment percentage for different outcomes of the previous marketing campaign (e.g. success, failure, no campaign)?"""
# query = """What percentage of customers contacted X times enroll for CDs, versus customers contacted Y times?"""

#api - product - weather
# query = """What is the weather like right now in New York City in degrees Farenheit?"""

#Response from Langchain
response =  run_query(query)
print("----------------------------------------------------------------------")
print(f'Q: {query}  \nA: {response}')

# Streamlit UI

### Write the Streamlit app

In [ ]:
import json
import boto3

import sqlalchemy
from sqlalchemy import create_engine
# from snowflake.sqlalchemy import URL

from langchain.docstore.document import Document
from langchain import PromptTemplate,SagemakerEndpoint,SQLDatabase, SQLDatabaseChain, LLMChain
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import SQLDatabaseSequentialChain

from langchain.chains.api.prompt import API_RESPONSE_PROMPT
from langchain.chains import APIChain
from langchain.prompts.prompt import PromptTemplate
from langchain.chat_models import ChatAnthropic
from langchain.chains.api import open_meteo_docs
from langchain.memory import ConversationBufferMemory

from typing import Dict
import time

import boto3
import streamlit as st
from streamlit_chat import message

glue_databucket_name = 'sagemaker-studio-741094476554-9zkt2s8krvb' #Create this bucket in S3
glue_db_name='ihmnick-bankadditional'
glue_role=  'ihmnick-AWSGlueServiceRole-glueworkshop120'
glue_crawler_name=glue_db_name+'-crawler120'


client = boto3.client('glue')
region=client.meta.region_name


# Connect to S3 using Athena
connathena=f"athena.{region}.amazonaws.com" 
portathena='443' #Update, if port is different
schemaathena=glue_db_name #from user defined params
s3stagingathena=f's3://{glue_databucket_name}/athenaresults/'#from cfn params
wkgrpathena='primary'#Update, if workgroup is different
# tablesathena=['dataset']#[<tabe name>]

# Create the athena connection string
connection_string = f"awsathena+rest://@{connathena}:{portathena}/{schemaathena}?s3_staging_dir={s3stagingathena}/&work_group={wkgrpathena}"

# Create the athena SQLAlchemy engine
engine_athena = create_engine(connection_string, echo=False)
dbathena = SQLDatabase(engine_athena)
gdc = [schemaathena] 

# Setup memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Generate Dynamic prompts to populate the Glue Data Catalog
# harvest aws crawler metadata
def parse_catalog():
    # Connect to Glue catalog
    # Get metadata of redshift serverless tables
    columns_str=''
    
    # Define glue cient
    glue_client = boto3.client('glue')
    
    for db in gdc:
        response = glue_client.get_tables(DatabaseName =db)
        for tables in response['TableList']:
            #classification in the response for s3 and other databases is different. Set classification based on the response location
            if tables['StorageDescriptor']['Location'].startswith('s3'):  classification='s3' 
            else:  classification = tables['Parameters']['classification']
            for columns in tables['StorageDescriptor']['Columns']:
                    dbname,tblname,colname=tables['DatabaseName'],tables['Name'],columns['Name']
                    columns_str=columns_str+f'\n{classification}|{dbname}|{tblname}|{colname}'                     
    # API
    # Append the metadata of the API to the unified glue data catalog
    columns_str=columns_str+'\n'+('api|meteo|weather|weather')
    print('columns_str', columns_str)
    return columns_str
        
glue_catalog = parse_catalog()

# Function 1 'Infer Channel'
# Define a function that infers the channel/database/table and sets the database for querying
def identify_channel(query):
    db = {}
    
    # Prompt 1 'Infer Channel'
    # Set prompt template. It instructs the llm on how to evaluate and respond to the llm. It is referred to as dynamic since glue data catalog is first getting generated and appended to the prompt.
    prompt_template = """
     From the table below, find the database (in column database) which will contain the data (in corresponding column_names) to answer the question 
     {query} \n
     """+glue_catalog +""" 
     Give your answer as database == 
     Also,give your answer as database.table == 
     """
    
    # Define prompt 1
    PROMPT_channel = PromptTemplate( template=prompt_template, input_variables=["query"]  )

    # define LLM chain
    llm_chain = LLMChain(prompt=PROMPT_channel, llm=llm)
    
    # Run the query and save to generated texts
    generated_texts = llm_chain.run(query)
    print('identified channel:', generated_texts)

    # Set the channel from where the query can be answered
    if 's3' in generated_texts: 
            channel='db'
            db=dbathena
            print("SET database to athena")
    elif 'api' in generated_texts: 
            channel='api'
            print("SET database to weather api")        
    else: 
        raise Exception("User question cannot be answered by any of the channels mentioned in the catalog")
    
    print("Step complete. Channel is: ", channel)
    
    return channel, db

# Define a function that infers the channel/database/table and sets the database for querying
def run_query(query):

    channel, db = identify_channel(query) #call the identify channel function first

    # Prompt 2 'Run Query'
    # After determining the data channel, run the Langchain SQL Database chain to convert 'text to sql' and run the query against the source data channel. 
    # provide rules for running the SQL queries in default template--> table info.

    _DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.

    Do not append 'Query:' to SQLQuery.
    
    Display SQLResult after the query is run in plain english that users can understand. 

    Provide answer in simple english statement.
 
    Only use the following tables:

    {table_info}

    Question: {input}"""

    PROMPT_sql = PromptTemplate(
        input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
    )

    if channel=='db':
        db_chain = SQLDatabaseChain.from_llm(llm, db, prompt=PROMPT_sql, verbose=True, return_intermediate_steps=False, memory=memory, use_query_checker=True)
        response = db_chain.run(query)
    elif channel == 'api':
        chain_api = APIChain.from_llm_and_api_docs(llm, open_meteo_docs.OPEN_METEO_DOCS, verbose=True, memory=memory)
        response = chain_api.run(query)
    else: 
        raise Exception("Unlisted channel. Check your unified catalog")
    
    return response

# Storing the chat
if 'generated' not in st.session_state:
    st.session_state['generated'] = []

if 'past' not in st.session_state:
    st.session_state['past'] = []
        
def action_search():
    st.title('Market Research Assistant')
    
    col1, col2 = st.columns(2)
    with col1:
        query = st.text_input('**Ask a question:**', '')
        button_search = st.button('Ask')
        
        if query or button_search:
            reply = run_query(query)
            # store the output 
            st.session_state.past.append(query)
            st.session_state.generated.append(reply)

        if st.session_state['generated']:
            for i in range(len(st.session_state['generated'])-1, -1, -1):
                message(st.session_state["generated"][i], key=str(i))
                message(st.session_state['past'][i], is_user=True, key=str(i) + '_user')
        
    # col1, col2 = st.columns(2)
    # with col1:
    #     query = st.text_input('**Ask a question:**', '')
    #     button_search = st.button('Ask')
    #     if query or button_search:
    #         message(query, is_user=True)
    #         response = run_query(query)
    #         # st.write(response)
    #         message(response)
                
def app_sidebar():
    with st.sidebar:
        st.write('## How to use:')
        description = """Assume the role of a marketing analyst working at a bank. There is an ask to perform analysis whether a customer will enroll for a certificate of deposit (CD). In order to perform the analysis, the marketing dataset contains information on customer demographics, responses to marketing events, and external factors.

- What percentage of customers in each age group enroll for CDs?
- What is the enrollment rate for each marital status?
- What percentage of customers in each education level enroll for CDs?
- What percentage of customers with existing loans (housing loan, personal loan) enroll for CDs?
- What is the average enrollment rate over time for each month/quarter?
- What is the average enrollment rate at different time periods after the last marketing contact (e.g. 0-7 days, 8-14 days, 15-30 days, 30+ days)?
- What is the enrollment percentage for different outcomes of the previous marketing campaign (e.g. success, failure, no campaign)?
- What percentage of customers contacted X times enroll for CDs, versus customers contacted Y times?"""
        st.write(description)
        st.write('---')


def main():
    st.set_page_config(layout="wide")
    app_sidebar()
    action_search()


if __name__ == '__main__':
    main()
    


## Start App

### Run Streamlit
To run the application:
1. Select File > New > Terminal
2. In the terminal, use the command: `streamlit run app_search.py --server.runOnSave true`
   1. Note: ensure you have installed all required packages
3. If this is successful, you will be able to interact with the app by using the web address below
4. An important thing to note is that when you run the above command, you should see an output similar to below.
5. The port thats  displayed is the same port that MUST be used after the `proxy` folder below.
`
You can now view your Streamlit app in your browser.

  Network URL: http://###.###.###.###:8501\
  External URL: http://###.###.###.###:8501



#### Display Link to Application

In [ ]:
import boto3
import sagemaker as sm
import json

def get_sagemaker_session(local_download_dir) -> sm.Session:
    """
    # Create a SageMaker Session
    # This function is used to create a SageMaker Session object.
    # The SageMaker Session object is used to create a SageMaker Endpoint,
    # SageMaker Model, and SageMaker Endpoint Config.
    """
    sagemaker_client = boto3.client(service_name="sagemaker", region_name=boto3.Session().region_name)
    session_settings = sm.session_settings.SessionSettings(local_download_dir=local_download_dir)
    session = sm.session.Session(sagemaker_client=sagemaker_client, settings=session_settings)
    return session

model_path = './download_dir'
if not os.path.exists(model_path):
    os.mkdir(model_path)
    
role               = sm.get_execution_role()
sagemaker_session  = get_sagemaker_session(model_path) # sm.session.Session()
region             = sagemaker_session._region_name

# These are needed to show where the streamlit app is hosted
sagemaker_metadata = json.load(open('/opt/ml/metadata/resource-metadata.json', 'r'))
domain_id          = sagemaker_metadata['DomainId']
resource_name      = sagemaker_metadata['ResourceName']

region = sagemaker_session._region_name

print(f'http://{domain_id}.studio.{region}.sagemaker.aws/jupyter/default/proxy/8501/')

### Clean-up
After you run the modern data architecture with Generative AI, make sure to clean up any resources that won’t be utilized. Delete the data in Amazon S3 and make sure to stop any SageMaker Studio notebook instances to not incur any further charges.
